### **1\. IntelliSense Frissítése**

Az IntelliSense néha nem tudja automatikusan frissíteni az aktív adatbázist. Kényszerítsd az IntelliSense újratöltését:

1. Nyisd meg a Command Palette-et (Ctrl+Shift+P vagy Cmd+Shift+P Mac-en).
2. Írd be: **"IntelliSense: Refresh IntelliSense Cache"**.
3. Válaszd ki az opciót, és várj néhány másodpercet.

Ez frissíti a táblaneveket és az aktuális adatbázis kontextust, ami gyakran megoldja az aláhúzások problémáját.

Íme egy **SQL** példa egy komplex adattranszformációra, amely magában foglalja több táblát, `JOIN`\-okat, `WINDOW` függvényeket, `TEMPORARY VIEW`\-kat és néhány **best practice** megoldást.

### Feladat

Van egy adatbázisunk az alábbi sémával:

1. **customers**: az ügyfelek adatai
    - `customer_id`, `name`, `signup_date`, `country`
2. **orders**: a rendelések adatai
    - `order_id`, `customer_id`, `order_date`, `total_amount`
3. **products**: a termékek adatai
    - `product_id`, `product_name`, `category`
4. **order\_details**: egy rendelésen belül a termékek részletei
    - `order_id`, `product_id`, `quantity`, `price_per_unit`

A cél:

- Készíts egy elemzést, amely megmutatja:
    1. Minden ügyfél összesített költését és a legutóbbi rendelésük időpontját.
    2. Az ügyfelek országonkénti összesített költését.
    3. A legnépszerűbb termékkategóriát az egyes ügyfelek esetében (legtöbbször rendelt kategória).

### SQL megoldás

#### Lépések:

1. Hozunk létre **TEMPORARY VIEW**\-kat a tranzformáció különböző szakaszaihoz, hogy a kód olvashatóbb és karbantarthatóbb legyen.
2. Az `INNER JOIN` segítségével összekapcsoljuk a táblákat.
3. `WINDOW FUNCTIONS`\-t használunk az ügyfelek és kategóriák aggregálására.
4. Alkalmazzuk a best practice-eket:
    - Aliasok használata.
    - Kommentek a kód fontos részeinél.
    - Modularitás (view-k).

### Magyarázat

1. **`TEMPORARY VIEW` használata**:
    
    - A különböző részfeladatokhoz létrehoztunk view-kat (`customer_orders`, `customer_order_details`, stb.). Ez modulárisabbá teszi a kódot, így könnyebb tesztelni és módosítani.
2. **`JOIN` műveletek**:
    
    - Az `INNER JOIN` kapcsolja össze az ügyfeleket a rendeléseikkel, majd a rendelési részleteket a termékekkel.
3. **`WINDOW FUNCTION` használata**:
    
    - A `RANK()` függvényt a kategóriák rangsorolására használtuk az ügyfelek számára, hogy megtaláljuk a legnépszerűbb kategóriát.
4. **Best practice-ek**:
    
    - `AS` aliasok használata az oszlopnevek és táblanevek egyértelműsítésére.
    - Kód modularizálása `VIEW`\-kkal, ami segíti az olvashatóságot és újrafelhasználhatóságot.
    - Az `LEFT JOIN`\-nal biztosítottuk, hogy minden ügyfél bekerüljön a végső riportba, még akkor is, ha nincs adat egyes csatlakoztatott táblákból.
5. **Final output**:  
    
    - Az ügyfelek költései, legutóbbi rendelési időpontja, top kategóriája, és országonkénti költések mind egy helyen jelennek meg.

In [3]:
-- CREATE DATABASE TestDatabase;
-- GO

USE [TestDatabase];  -- Switch to the desired database
GO



-- Verify current database
SELECT DB_NAME() AS CurrentDatabase;


: Msg 319, Level 15, State 1, Line 4
Incorrect syntax near the keyword 'with'. If this statement is a common table expression, an xmlnamespaces clause or a change tracking context clause, the previous statement must be terminated with a semicolon.

(1 row affected)

Total execution time: 00:00:00.028

CurrentDatabase
TestDatabase


## CTE - Common Table Expressions

In [24]:
--CTE Common Table Expressions

WITH CustomerTotals AS (
    SELECT 
        c.customer_id, 
        c.name, 
        SUM(o.total_amount) AS TotalSpent
    FROM [TestDatabase].[dbo].[customers] c
    LEFT JOIN [TestDatabase].[dbo].orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name
)
SELECT *
FROM CustomerTotals
WHERE TotalSpent > 5000;


SELECT 
    o.customer_id,
    o.order_date,
    o.total_amount,
    SUM(o.total_amount) OVER (PARTITION BY o.customer_id ORDER BY o.order_date) AS CumulativeSpend
FROM [TestDatabase].[dbo].[orders] o;


(0 rows affected)

(5 rows affected)

Total execution time: 00:00:00.043

customer_id,name,TotalSpent


customer_id,order_date,total_amount,CumulativeSpend
1,2024-01-10,2000,2000
1,2024-01-15,3000,5000
2,2024-01-12,3000,3000
3,2024-01-14,1500,1500
3,2024-01-16,500,2000


In [33]:
--CTE Common Table Expressions

WITH CustomerTotals AS (
    SELECT 
        c.customer_id, 
        c.name, 
        SUM(o.total_amount) AS TotalSpent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name
)
SELECT *
FROM CustomerTotals
WHERE TotalSpent > 5000;


SELECT 
    o.customer_id,
    o.order_date,
    o.total_amount,
    SUM(o.total_amount) OVER (PARTITION BY o.customer_id ORDER BY o.order_date) AS CumulativeSpend
FROM orders o;

(0 rows affected)

(5 rows affected)

Total execution time: 00:00:00.053

customer_id,name,TotalSpent


customer_id,order_date,total_amount,CumulativeSpend
1,2024-01-10,2000,2000
1,2024-01-15,3000,5000
2,2024-01-12,3000,3000
3,2024-01-14,1500,1500
3,2024-01-16,500,2000


## Stored Procedures

In [28]:
-- stored procedure
-- DROP PROCEDURE GetCustomerOrders;

CREATE PROCEDURE GetCustomerOrders
    @CustomerID INT
AS
BEGIN
    SELECT * 
    FROM [TestDatabase].[dbo].[orders] 
    WHERE customer_id = @CustomerID;
END;


: Msg 2714, Level 16, State 3, Procedure GetCustomerOrders, Line 4
There is already an object named 'GetCustomerOrders' in the database.

Total execution time: 00:00:00.023

In [29]:
-- Call a procedure
EXEC GetCustomerOrders @CustomerID = 1;

(2 rows affected)

Total execution time: 00:00:00.010

order_id,customer_id,order_date,total_amount
101,1,2024-01-10,2000
103,1,2024-01-15,3000


In [8]:
-- 1. Ügyfelek rendeléseinek részletei
CREATE OR ALTER VIEW customer_orders AS
SELECT 
    c.customer_id,
    c.name AS customer_name,
    c.country,
    o.order_id,
    o.order_date,
    o.total_amount
FROM 
    [dbo].[customers] c
INNER JOIN 
    [dbo].[orders] o 
ON 
    c.customer_id = o.customer_id;

Commands completed successfully.

Total execution time: 00:00:00.049

In [10]:
-- 2. Termékek és rendelési részletek hozzáadása
CREATE OR ALTER VIEW customer_order_details AS
SELECT 
    co.customer_id,
    co.customer_name,
    co.country,
    co.order_id,
    co.order_date,
    co.total_amount,
    od.product_id,
    od.quantity,
    od.price_per_unit,
    p.product_name,
    p.category
FROM 
    [dbo].[customer_orders] co
INNER JOIN 
    [dbo].order_details od 
ON 
    co.order_id = od.order_id
INNER JOIN 
    [dbo].[products] p 
ON 
    od.product_id = p.product_id;

Commands completed successfully.

Total execution time: 00:00:00.021

In [11]:
-- 3. Ügyfelek összesített költése és legutóbbi rendelési időpontja
CREATE OR ALTER VIEW customer_spending AS
SELECT 
    customer_id,
    customer_name,
    country,
    SUM(total_amount) AS total_spent,
    MAX(order_date) AS last_order_date
FROM 
    customer_orders
GROUP BY 
    customer_id, customer_name, country;

Commands completed successfully.

Total execution time: 00:00:00.025

In [12]:
-- 4. Országonkénti összesített költés
CREATE OR ALTER VIEW country_spending AS
SELECT 
    country,
    SUM(total_amount) AS total_country_spent,
    COUNT(DISTINCT customer_id) AS total_customers
FROM 
    customer_orders
GROUP BY 
    country;

Commands completed successfully.

Total execution time: 00:00:00.010

In [15]:
-- 5. Legnépszerűbb termékkategória ügyfelenként
CREATE OR ALTER VIEW popular_categories AS
SELECT 
    customer_id,
    customer_name,
    category,
    SUM(quantity) AS total_quantity,
    RANK() OVER (PARTITION BY customer_id ORDER BY SUM(quantity) DESC) AS rank_category
FROM 
    customer_order_details
GROUP BY 
    customer_id, customer_name, category;

Commands completed successfully.

Total execution time: 00:00:00.012

In [16]:
-- Csak a legnépszerűbb kategóriák
CREATE OR ALTER VIEW top_categories AS
SELECT 
    customer_id,
    customer_name,
    category AS top_category
FROM 
    popular_categories
WHERE 
    rank_category = 1;

Commands completed successfully.

Total execution time: 00:00:00.010

In [17]:
-- 6. Final Report
SELECT 
    cs.customer_id,
    cs.customer_name,
    cs.country,
    cs.total_spent,
    cs.last_order_date,
    tc.top_category,
    cs1.total_country_spent
FROM 
    customer_spending cs
LEFT JOIN 
    top_categories tc
ON 
    cs.customer_id = tc.customer_id
LEFT JOIN 
    country_spending cs1
ON 
    cs.country = cs1.country
ORDER BY 
    cs.total_spent DESC;


(3 rows affected)

Total execution time: 00:00:00.083

customer_id,customer_name,country,total_spent,last_order_date,top_category,total_country_spent
1,John Doe,USA,5000,2024-01-15,Electronics,8000
2,Jane Smith,USA,3000,2024-01-12,Books,8000
3,Alice Brown,UK,2000,2024-01-16,Home Appliances,2000


In [18]:
DECLARE @MyVariable INT;
SET @MyVariable = 10;


Commands completed successfully.

Total execution time: 00:00:00.004

In [22]:
DECLARE @CustomerName NVARCHAR(100);
SELECT @CustomerName = name FROM dbo.customers WHERE customer_id = 1;



Commands completed successfully.

Total execution time: 00:00:00.006